In [75]:
import pandas as pd
import json
import re
from collections import defaultdict

In [80]:
class ABSReader:
    
    def __init__(self, subj='Total Personal Income'):
        
        self.table_names = pd.read_excel('/Users/ik/Data/2016_GCP_POA_for_NSW_short-header/Metadata/Metadata_2016_GCP_DataPack.xlsx', 
                     skiprows=9, sheet_name=0)

        self.what_tables = list(self.table_names[self.table_names['Table name'].apply(lambda x: subj in x)]['Table number'])

        self.cell_descr = pd.read_excel('/Users/ik/Data/2016_GCP_POA_for_NSW_short-header/Metadata/Metadata_2016_GCP_DataPack.xlsx', 
                     skiprows=10, sheet_name=1)

        self.dsc = self.cell_descr[self.cell_descr['Profile table'].apply(lambda x: sum([table_name in x for table_name in self.what_tables]) > 0 )]

        fls_cont_ = list(self.dsc['DataPack file'].unique())

        self.age_groups = []

        for lab in list(self.dsc['Short']):
            _ = re.search(r'_\d{2}_',lab)
            if _:
                self.age_groups.append(_.group(0)[:-1])
                
        self.age_groups = sorted(list(set(self.age_groups)))  # ['_15', '_20', '_25',...
        
        self.tbl = pd.concat([pd.read_csv(f'/Users/ik/Data/2016_GCP_POA_for_NSW_short-header/2016 Census GCP Postal Areas for NSW/2016Census_{t}_NSW_POA.csv').set_index('POA_CODE_2016') for t in fls_cont_], axis=1)
        
        self.income_dict = defaultdict()
        

    def is_income_col(self, col_name):
        
        gender, dollars_from, dollars_to = col_name.split('_')[:3]
        
        if not ((dollars_from.lower() in {'neg', 'negtve', 'pi'}) or dollars_from.isdigit()):
            return (None, None)
        
        if dollars_from.lower() in {'neg', 'negtve'}:
            dollars_from = '0'
        elif dollars_from.lower() == 'pi':
            dollars_from = '?'
            
        if dollars_to.lower() == 'nil':
            dollars_to = ''
        elif dollars_to.lower() == 'ns':
            dollars_to = ''
        elif dollars_to.lower() == 'more':
            dollars_to = '+'
        
        income = f'${dollars_from}{"_" if dollars_to.isdigit() else ""}{dollars_to}'
            
        return (gender.lower(), income) 
    
    def js(self):
        
        for pcode in self.tbl.index:
            _pcode = pcode[3:]
            if _pcode not in self.income_dict:
                self.income_dict[_pcode] = defaultdict()
            for g in ['m', 'f']:
                if g not in self.income_dict[_pcode]:
                    self.income_dict[_pcode][g] = defaultdict()
                for age in self.age_groups:
                    _age = age.replace('_','')
                    if _age != '85':
                        yrs = 4 if int(_age) < 25 else 9
                        _age = f'{_age}_{str(int(_age) + yrs)}'
                    else:
                        _age += '+'
                    if _age not in self.income_dict[_pcode][g]:
                        self.income_dict[_pcode][g][_age] = defaultdict()
                    _ = self.tbl[self.tbl.index == pcode]
                    _ = _[[c for c in _.columns if (age in c) and (g.upper() in c)]]
                    dk = _.to_dict(orient='index')[pcode]
                    for v in dk:
                        _g, inc = self.is_income_col(v)
                        if _g:
                            self.income_dict[_pcode][g][_age][inc] = dk[v] 
        return self
    
    def save(self, fl):
        
        json.dump(self.income_dict, open(fl, 'w'))

In [81]:
if __name__ == '__main__':
    
    a = ABSReader().js().save('abs_personal_income_by_age_gender_weekly_nsw.json')